In [1]:
# import arcpy
import os
import sys
import fnmatch
import itertools
import numpy as np
from osgeo import gdal
#import rasterio
import xml.etree.ElementTree as et

In [2]:
#constant parameters 
pi = 3.14159265358
offset = {'BAND_R': -2.512, 'BAND_G': -3.546, 'BAND_B': -5.736, 'BAND_N': -3.300}
gain = {'BAND_R':0.952, 'BAND_G': 0.36, 'BAND_B':0.988, 'BAND_N':0.961}
esun = {'BAND_R': 1538.85, 'BAND_G':1829.62,'BAND_B':2007.27, 'BAND_N':1053.21}


In [3]:
folder = r'C:\\Toar\\NGA_Ngala_WV2_12Aug2016_2016_55_249_original\\055795286010_01\\055795286010_01_P001_MUL\\' # path to the P001_MUL 
#get image and metadata paths
def find_paths(folder):
    image =[]
    metadata_path =[]  
    #arcpy.env.workspace = r'C:\Toar'= wkspc
    for k in os.listdir(folder):
        if k.endswith('.XML'):
            metadata_path = os.path.join(folder,k)
        if k.endswith('.TIF'):
            image = os.path.join(folder,k)
                #dataset = gdal.Open(image)
    return(metadata_path, image)

 # #get band info
# bandarr = dataset.GetRasterBand(1).ReadAsArray()
# proj = dataset.GetProjection()
# transf = dataset.GetGeoTransform()
# # xsize = dataset.RasterXSize
# ysize = dataset.RasterYSize
# # bandarr = bandarr.astype('f')
# format1 = 'GTiff' #change to 'ENVI'
# driver = gdal.GetDriverByName(format1)
# # dta_type = gdal.GDT_Float32



m_path, i_path = find_paths(folder)

print(m_path)
print(i_path)

C:\\Toar\\NGA_Ngala_WV2_12Aug2016_2016_55_249_original\\055795286010_01\\055795286010_01_P001_MUL\\16AUG12093434-M2AS-055795286010_01_P001.XML
C:\\Toar\\NGA_Ngala_WV2_12Aug2016_2016_55_249_original\\055795286010_01\\055795286010_01_P001_MUL\\16AUG12093434-M2AS-055795286010_01_P001.TIF


In [4]:
#read neccesary parameters from the metadata
def find_metad(metadata_path):
    #iterate through the metada
    tree = et.parse(metadata_path)
    root = tree.getroot()

    #uncomment to see the tag for each band (but not neccesary)
    # [elem.tag for elem in root.iter()]

    # extract the abscalfactor and effectivebandwidth 
    band_names = ['BAND_R', 'BAND_G','BAND_B', 'BAND_N'] #picked from the metadata
    g=[]
    for name in band_names:
        for b in root.iter(name):
            for b,c in itertools.product(b.iter('EFFECTIVEBANDWIDTH'), b.iter('ABSCALFACTOR')):
                d =float(c.text)/float(b.text) #(abscalfacto/effective bandwidth) --- see equation 1
                d = round(d,7)
                g.append(d)
    band_factors = dict(zip(band_names,g)) 

    return band_factors

absl = find_metad(m_path)

In [5]:
print(absl)

{'BAND_R': 0.3226019, 'BAND_G': 0.2165392, 'BAND_B': 0.3284656, 'BAND_N': 0.2073638}


## L = Gain x DN x (abscalfactor/effective bandwidth) + Offset  ------Equation 1


In [6]:
#read the image .TIF from the folder 
dataset = gdal.Open(i_path, gdal.GA_ReadOnly)

# selecting each band  and read as array, might not be the best approach but I thought it would work
Band_R = dataset.GetRasterBand(1).ReadAsArray()
Band_G = dataset.GetRasterBand(2).ReadAsArray()
Band_B = dataset.GetRasterBand(3).ReadAsArray()
Band_N = dataset.GetRasterBand(4).ReadAsArray()

#passed all bands into array for easily looping, but it's not!!!
layer = [Band_R, Band_G, Band_B, Band_N]

#From the equation 1, below it meant to implement Equation 1. It does work but could not perform further arithmetric operations on the output array - c
for a,b,c,d in zip(layer,absl.values(), offset.values(),gain.values()):
    L = d*np.multiply(a,b)+c # I know thhis look rough but I ju
    print(L) #Is it possile to extract each bands from L using index? I tried didn't work

[[ -2.512       -2.512       -2.512      ...  -2.512       -2.512
   -2.512     ]
 [ -2.512       -2.512       -2.512      ...  -2.512       -2.512
   -2.512     ]
 [ -2.512       -2.512       -2.512      ...  -2.512       -2.512
   -2.512     ]
 ...
 [ -2.512       -2.512       -2.512      ... 113.88534634 110.19994223
   -2.512     ]
 [ -2.512       -2.512       -2.512      ... 106.51453812 106.20742112
   -2.512     ]
 [ -2.512       -2.512       -2.512      ...  -2.512       -2.512
   -2.512     ]]
[[-3.546      -3.546      -3.546      ... -3.546      -3.546
  -3.546     ]
 [-3.546      -3.546      -3.546      ... -3.546      -3.546
  -3.546     ]
 [-3.546      -3.546      -3.546      ... -3.546      -3.546
  -3.546     ]
 ...
 [-3.546      -3.546      -3.546      ... 43.53828365 42.83669664
  -3.546     ]
 [-3.546      -3.546      -3.546      ... 40.42011917 40.18625683
  -3.546     ]
 [-3.546      -3.546      -3.546      ... -3.546      -3.546
  -3.546     ]]
[[ -5.736       -5.7

In [8]:
L[1] #dosen't look good

array([-3.3, -3.3, -3.3, ..., -3.3, -3.3, -3.3])

In [9]:
#I was just trying if working on each band is a good idea.
radiance_R = gain['BAND_R']* Band_R*absl['BAND_R']+offset['BAND_R']
radiance_R

array([[ -2.512     ,  -2.512     ,  -2.512     , ...,  -2.512     ,
         -2.512     ,  -2.512     ],
       [ -2.512     ,  -2.512     ,  -2.512     , ...,  -2.512     ,
         -2.512     ,  -2.512     ],
       [ -2.512     ,  -2.512     ,  -2.512     , ...,  -2.512     ,
         -2.512     ,  -2.512     ],
       ...,
       [ -2.512     ,  -2.512     ,  -2.512     , ..., 113.88534634,
        110.19994223,  -2.512     ],
       [ -2.512     ,  -2.512     ,  -2.512     , ..., 106.51453812,
        106.20742112,  -2.512     ],
       [ -2.512     ,  -2.512     ,  -2.512     , ...,  -2.512     ,
         -2.512     ,  -2.512     ]])

In [10]:
 #this  needed to write out each bands
 #get band info
band_ds = gdal.Open(i_path)

# proj = band_ds.GetProjection()
transf = band_ds.GetGeoTransform()
xsize = band_ds.RasterXSize
ysize = band_ds.RasterYSize
# band  = band.astype('f')
format1 = 'GTiff' #change to 'ENVI'
driver = gdal.GetDriverByName(format1)
dta_type = gdal.GDT_Float32